<a href="https://colab.research.google.com/github/somanshu1/Multi_agent_with_route/blob/main/Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U datasets huggingface_hub fsspec
!pip install -q langchain langchain-google-genai google-generativeai datasets numpy sentence-transformers pycoingecko transformers chromadb langchain-huggingface huggingface-hub
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 19.2 MB/s eta

In [ ]:
import os
import logging
import getpass
import numpy as np
from typing import Dict, Any
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from pycoingecko import CoinGeckoAPI
import chromadb
from chromadb.utils import embedding_functions

In [ ]:
# LangChain imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable import Runnable

In [ ]:
print("Please enter your API keys:")
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your GOOGLE_API_KEY: GEMINI_API_KEY")
#os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")
#os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your Hugging Face API Token: ")

Please enter your API keys:
Enter your GOOGLE_API_KEY: GEMINI_API_KEY··········


In [ ]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s — %(levelname)s — %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
logger.info("Loading dataset…")
dataset = load_dataset("nvidia/HelpSteer3", "preference", split="train")

logger.info("Initializing embedding model…")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

README.md: 0.00B [00:00, ?B/s]

train.jsonl.gz:   0%|          | 0.00/106M [00:00<?, ?B/s]

validation.jsonl.gz:   0%|          | 0.00/5.49M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/38459 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2017 [00:00<?, ? examples/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Step 5: Initialize ChromaDB for high-performance vector storage
logger.info("Setting up ChromaDB vector database...")

# Create an in-memory ChromaDB client
chroma_client = chromadb.Client()

# Clear any old collection to ensure a fresh start
try:
    chroma_client.delete_collection(name="helpsteer_prompts")
    logger.info("Cleared old ChromaDB collection to ensure fresh data.")
except Exception:
    pass # Ignore error if the collection didn't exist

try:
    collection = chroma_client.get_collection(name="helpsteer_prompts")
    logger.info("ChromaDB collection 'helpsteer_prompts' already exists and is ready.")

except Exception:
    logger.info("Creating and populating new collection 'helpsteer_prompts'.")
    collection = chroma_client.create_collection(
        name="helpsteer_prompts",
        metadata={"hnsw:space": "cosine"}
    )

    batch_size = 5000
    total_samples = len(dataset)
    logger.info(f"Processing {total_samples} prompts in batches of {batch_size}...")

    for i in range(0, total_samples, batch_size):
        batch_end = min(i + batch_size, total_samples)
        batch_prompts = dataset[i:batch_end]['context']

        # --- FINAL FIX: Convert every item to a string to prevent IndexError ---
        # This handles cases where 'context' contains lists, None, or other non-string data.
        sanitized_prompts = [str(p) for p in batch_prompts]

        # Use the sanitized list for encoding and storage
        batch_embeddings = embedding_model.encode(sanitized_prompts, convert_to_numpy=True)
        ids = [f"prompt_{j}" for j in range(i, batch_end)]
        embeddings = batch_embeddings.tolist()
        metadatas = [{"prompt": p} for p in sanitized_prompts]
        documents = sanitized_prompts

        collection.add(
            ids=ids,
            embeddings=embeddings,
            metadatas=metadatas,
            documents=documents
        )
        logger.info(f"Loaded batch {i//batch_size + 1}/{(total_samples + batch_size - 1)//batch_size}")

    logger.info("Successfully loaded all embeddings into ChromaDB!")

In [ ]:
# Step 6: Implement the Heaviness Router (Corrected)
class HeavinessRouter:
    """
    Router that determines prompt complexity by querying similar prompts in the vector database.
    Uses ChromaDB for high-performance similarity search.
    """

    def __init__(self, collection, embedding_model, threshold=0.7):
        self.collection = collection
        self.embedding_model = embedding_model
        self.threshold = threshold

    def calculate(self, prompt: str) -> bool:
        """
        Calculate if a prompt is 'heavy' based on similarity to existing prompts.
        """
        # Generate embedding for the input prompt
        prompt_embedding = self.embedding_model.encode([prompt], convert_to_numpy=True)

        # Query ChromaDB for the top 10 most similar prompts
        results = self.collection.query(
            query_embeddings=prompt_embedding.tolist(),
            n_results=10
        )

        distances = results['distances'][0]

        # --- FIX: Handle cases where the query returns no results ---
        if not distances:
            logger.warning("Vector database query returned no results. Defaulting to HEAVY task.")
            return True # Default to heavy if no similar prompts are found

        # Calculate average similarity (cosine similarity = 1 - distance)
        similarities = [1 - dist for dist in distances]
        avg_similarity = np.mean(similarities)

        # Determine if prompt is heavy (low similarity to existing prompts)
        is_heavy = avg_similarity < self.threshold

        logger.info(f"Average similarity: {avg_similarity:.4f}")
        logger.info(f"Routing decision: {'HEAVY' if is_heavy else 'LIGHT'} (threshold: {self.threshold})")

        return is_heavy

In [ ]:
# Step 7: Initialize the llm
from langchain_community.llms import HuggingFaceHub
from langchain_google_genai import ChatGoogleGenerativeAI
router = HeavinessRouter(collection, embedding_model, threshold=0.7)
# In Step 8 of the notebook

logger.info("Initializing language models...")

# Light LLM - Fast and cost-effective Gemini model
light_llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.3
)

# Heavy LLM - More powerful Gemini model for complex tasks
heavy_llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7
)
'''from langchain.chat_models import ChatOpenAI

# Light LLM - Fast and cost-effective OpenAI model
light_llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.3
)

# Heavy LLM - More powerful OpenAI model for complex tasks
heavy_llm = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0.7
)
'''
'''# Light LLM - Zephyr 7B (chat-tuned)
light_llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    model_kwargs={"temperature": 0.3, "max_new_tokens": 512}
)

# Heavy LLM - Mistral 7B Instruct (more verbose & strong reasoning)
heavy_llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    model_kwargs={"temperature": 0.7, "max_new_tokens": 512}
)'''

logger.info("Models initialized: light_llm=gemini-1.5-flash, heavy_llm=gemini-1.5-pro")

In [ ]:
# In Step 9 of the notebook

# --- FINAL FIX: Map common names to the correct API ID ---
TOKEN_ID_MAP = {
    "bitcoin": "bitcoin",
    "btc": "bitcoin",
    "ethereum": "ethereum",
    "eth": "ethereum",
    "solana": "solana",
    "sol": "solana",
    "cardano": "cardano",
    "ada": "cardano",
    "dogecoin": "dogecoin",
    "doge": "dogecoin",
}

# Initialize the client ONCE, globally.
cg = CoinGeckoAPI()

def get_crypto_price(token_id: str) -> str:
    """Fetches the current USD price for a given CoinGecko token ID."""
    try:
        price_data = cg.get_price(ids=token_id, vs_currencies='usd')
        if price_data and token_id in price_data:
            price = price_data[token_id]['usd']
            # Capitalize the token name for cleaner output
            return f"The current price of {token_id.capitalize()} is ${price:,.2f} USD"
        else:
            return f"Could not find price data for {token_id}"
    except Exception as e:
        logger.error(f"Error fetching crypto price for {token_id}: {str(e)}")
        return f"Error fetching price for {token_id}"

In [ ]:
# In Step 10 of the notebook

def process_and_route(input_dict: Dict[str, Any]) -> Dict[str, Any]:
    """
    Applies rule-based routing for simple queries and falls back to the
    semantic router for complex ones.
    """
    prompt = input_dict.get("prompt", "")
    final_prompt = prompt
    crypto_keywords = ["price", "cost", "worth", "value"]

    # Use the keys from our map for detection
    mentioned_token_key = next((token for token in TOKEN_ID_MAP.keys() if token in prompt.lower()), None)
    is_price_query = any(keyword in prompt.lower() for keyword in crypto_keywords)

    if is_price_query and mentioned_token_key:
        logger.info("Rule-based route: Detected a simple price query.")

        # Look up the correct API ID from the map
        token_id = TOKEN_ID_MAP[mentioned_token_key]
        price_info = get_crypto_price(token_id) # Pass the correct ID

        final_prompt = f"The user asked: '{prompt}'. Provide a direct answer using this live data: {price_info}"

        logger.info("Forcing route to LIGHT LLM (Gemini 1.5 Flash) for this task.")
        llm_to_use = light_llm
        llm_name = "Light (Gemini 1.5 Flash)"

    else:
        # If it's not a price query, fall back to the semantic router
        logger.info("No rule matched. Using semantic heaviness router.")
        is_heavy = router.calculate(prompt)

        if is_heavy:
            logger.info("Routing to HEAVY LLM (Gemini 1.5 Pro)")
            llm_to_use = heavy_llm
            llm_name = "Heavy (Gemini 1.5 Pro)"
        else:
            logger.info("Routing to LIGHT LLM (Gemini 1.5 Flash)")
            llm_to_use = light_llm
            llm_name = "Light (Gemini 1.5 Flash)"

    return {"llm": llm_to_use, "llm_name": llm_name, "final_prompt": final_prompt}

In [ ]:
# In Step 11 of the notebook

# This new function calls the LLM and packages the output with the LLM's name
def invoke_and_package_response(inputs: Dict[str, Any]) -> Dict[str, str]:
    llm = inputs["llm"]
    llm_name = inputs["llm_name"]
    final_prompt = inputs["final_prompt"]

    # Invoke the LLM
    response = llm.invoke(final_prompt)
    response_text = response.content if hasattr(response, 'content') else str(response)

    return {"response": response_text, "source": llm_name}

# CHANGED: The pipeline now uses the new functions
pipeline = RunnableLambda(process_and_route) | RunnableLambda(invoke_and_package_response)

In [ ]:
# In your final UI cell

import traceback
from ipywidgets import VBox, Textarea, Button, Output, Label
from IPython.display import display
from google.api_core import exceptions # <-- ADD THIS IMPORT

# (Widget creation code remains the same)
prompt_input = Textarea(value='', placeholder='Enter your prompt here...', description='Prompt:', layout={'width': '100%', 'height': '100px'})
send_button = Button(description="Send", button_style="primary")
response_output = Output(layout={'border': '1px solid gray', 'padding': '10px'})
status_label = Label(value="")

# Callback logic with improved error handling
def on_send(btn):
    with response_output:
        response_output.clear_output()
        status_label.value = "Processing..."
        inp = {'prompt': prompt_input.value}
        try:
            out = pipeline.invoke(inp)
            source = out.get("source", "Unknown LLM")
            text = out.get("response", "No response.")
            print(f"Source: {source}\n---\n{text}")

        # --- CATCH THE SPECIFIC RATE LIMIT ERROR ---
        except exceptions.ResourceExhausted as e:
            print("⚠️ API Rate Limit Exceeded.")
            print("You've made too many requests on the free tier. Please wait a minute and try again.")

        except Exception as e:
            print("⚠️ An unknown error occurred. See details below:\n")
            traceback.print_exc()

        status_label.value = "Done."

send_button.on_click(on_send)

ui = VBox([prompt_input, send_button, status_label, response_output])
display(ui)